In [ ]:
# File: domino.ipynb
# Author: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
import numpy as np
from frontend import App

app = App.create()
V, F, T = app.mesh.box().tetrahedralize().scale(0.1, 0.25, 0.025)
app.asset.add.tet("block", V, F, T)

scene = app.scene.create("domino")

R, minR, d, N, C = 1.0, 0.5, 0.15, 4096, 2
xz = None
for i in reversed(range(N)):
    t = 2.0 * C * np.pi * i / N - np.pi / 2
    r = (R - minR) * i / N + minR
    angle = 180 * t / np.pi
    x, z = -r * np.cos(t), r * np.sin(t)
    if xz is None or np.linalg.norm(np.array([x, z]) - xz) > d:
        scene.add("block").at(x, 0, z).rotate(angle, "y")
        xz = np.array([x, z])

scene.add("block").at(-0.1, 0.25, -R).rotate(90, "y").rotate(-20, "z")

gap = app.session.param().get("contact-ghat")
scene.add.invisible.wall([0, scene.min("y") - 0.5 * gap, 0], [0, 1, 0])

fixed = scene.build().report()
opts = {"flat_shading": True, "wireframe": True}
fixed.preview(options=opts)

In [ ]:
param = app.session.param()
(
    param.set("volume-poiss-rat", 0.49)
    .set("friction", 0.1)
    .set("min-newton-steps", 32)
    .set("dt", 0.01)
    .set("fps", 30)
    .set("frames", 250)
)

session = app.session.create(fixed)
session.start(param).preview(options=opts)
session.stream()

In [ ]:
session.animate(options=opts)

In [ ]:
session.export.animation()

In [ ]:
# this is for CI
assert session.finished()

In [ ]:
if not app.ci:
    # An increased stiffness
    param.set("volume-poiss-rat", 0.499).set("frames", 32).set("barrier", "cubic")
    stiff_cubic_session = app.session.create(fixed)
    stiff_cubic_session.start(param).preview(options=opts)
    stiff_cubic_session.stream()

In [ ]:
if not app.ci:
    stiff_cubic_session.animate(options=opts)

In [ ]:
if not app.ci:
    # An increased stiffness quadratic
    param.set("barrier", "quad")
    stiff_quad_session = app.session.create(fixed)
    stiff_quad_session.start(param).preview(options=opts)
    stiff_quad_session.stream()

In [ ]:
if not app.ci:
    stiff_quad_session.animate(options=opts)